In [2]:
import json
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [3]:
def get_requirements_from_text(full_text: str):
    prompt = f"""
당신은 회의록에서 **시스템 개발자 관점에서 직접적으로 필요한 요구사항**을 추출하고 구조화하는 전문가입니다.  
각 요구사항은 시스템의 설계, 개발, 테스트, 배포, 운영에 직접적으로 활용될 수 있어야 하며, **요구사항의 상태(`status`)가 '신규', '변경', '삭제' 중 하나로 반드시 분류**되어야 합니다.

---

🧠 **요구사항 상태(status) 분류 기준**:

1. ✅ **신규**
   - 회의록에서 처음 언급된 내용이며, 이전에 존재하지 않았던 기능/조건/요구
   - 또는 기존 시스템에 없던 항목 추가 요청

2. 🔁 **변경**
   - 기존 요구사항을 **보완하거나 수정**하는 지시
   - 예: "로그인 방식에 생체 인증 추가", "응답 시간 기준을 5초에서 3초로 변경"

3. ❌ **삭제**
   - 다음 표현이 포함되면 무조건 '삭제'로 간주:
     - `"삭제"`, `[삭제]`, "더 이상 사용하지 않음", "제거", "폐기", "적용 안 함", "필요 없음", "제외", "미반영 예정"
   - 또는 기능이 중복되거나, 무의미하다는 언급이 있는 경우
   - 예: "이 기능은 이미 다른 모듈에서 처리하므로 제거함", "비효율적이라 제외 예정"

**주의:** 단순히 언급이 없거나, 기능 유사성이 있다고 해서 기존 항목이 '변경'으로 처리되면 안 됩니다. 명확히 "삭제"된다는 표현이 있거나 기능 제거가 언급된 경우에만 삭제로 분류해야 합니다.

---

📝 **요구사항 JSON 구조**:

각 요구사항은 아래 속성을 포함하는 JSON 객체입니다:

- **id**: 고유 식별자 (예: "FUNC-001", "NFR-001"). RFP 원문에 ID가 있다면 그것을 우선 사용
- **type**: 요구사항 유형: "기능적", "비기능적"
- **description**: 요구사항 요약 (시스템이 수행해야 하는 기능, 조건 등)
- **detailed_description**: 상세 설명
- **acceptance_criteria**: 요구사항 충족 여부를 판단할 수 있는 구체적 기준 (없으면 "세부 설계 시 정의"로 표기)
- **module**: 관련 시스템 모듈 (예: "인증 모듈", "전체 시스템"). 불분명하면 "미정"
- **source_pages**: 원본 파일명
- **raw_text_snippet**: 추출 근거가 된 원문 텍스트 조각
- **status**: "신규", "변경", "삭제" 중 하나만 선택
- **mod_reason**: 해당 상태로 분류된 이유. 예:
  - 신규: "회의 내용을 기반으로 신규 도출됨"
  - 변경: "기존 요구사항 조건 수정 지시"
  - 삭제: "회의에서 해당 기능 제거 결정"
- **category_large** / **category_medium** / **category_small**: 기능 분류 (예: "보안" / "접근 제어" / "권한 등급 설정")
- **difficulty**: 구현 난이도 ("상", "중", "하")
- **importance**: 시스템 성공에 대한 영향도 ("상", "중", "하")

---

⚠️ 주의사항:
- 출력은 반드시 위와 같은 구조의 **JSON 배열**이어야 하며, 최상위 키 없이 배열로만 출력하십시오.
- 회의록 내용 중 다음 항목은 요구사항으로 간주하지 말고 무시하십시오:
  - 목차
  - 제안서 작성 요령
  - 업체 선정 기준
  - 일반적인 계약 조건
  - 시스템 구현과 직접 관련 없는 행정적, 절차적 설명
- 요구사항이 존재하지 않으면 빈 배열 `[]`을 그대로 출력하십시오.

다음은 회의록 텍스트입니다. 여기서 직접적인 요구사항을 추출하십시오:

\"\"\"
{full_text}
\"\"\"
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 시스템 분석 전문가이며, 요구사항을 추출하는 역할입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return json.dumps({"error": str(e)})


In [4]:
from docx import Document
import pandas as pd
from pypdf import PdfReader

In [5]:
input_txt = "./data/대한체육회_회의록.txt"
input_pdf = "./data/대한체육회_학습운영시스템_추가_요구사항_제안_회의_텍스트_변환본.pdf"
input_docs = "./data/대한체육회 학습운영시스템 추가 요구사항 제안 회의 텍스트 변환본.docx"
input_xlsx = "./data/대한체육회 학습운영시스템 추가 요구사항 제안 회의 텍스트 변환본.xlsx"

In [6]:
def read_docx(file_path: str) -> str:
    doc = Document(file_path)
    full_text = [para.text for para in doc.paragraphs]
    return "\n".join(full_text)

def read_pdf(file_path: str) -> str:
    reader = PdfReader(file_path)
    texts = []
    for page in reader.pages:
        texts.append(page.extract_text() or "")
    return "\n".join(texts)

def read_excel(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".xls":
        df = pd.read_excel(file_path, engine='xlrd')  # .xls 전용
    else:
        df = pd.read_excel(file_path, engine='openpyxl')  # .xlsx 전용
    text = df.astype(str).agg(' '.join, axis=1).str.cat(sep='\n')
    return text

def read_text_or_pdf_or_docx_or_excel(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".txt":
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    elif ext == ".pdf":
        return read_pdf(file_path)
    elif ext == ".docx":
        return read_docx(file_path)
    elif ext in [".xls", ".xlsx"]:
        return read_excel(file_path)
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. txt, pdf, docx, xls, xlsx만 지원합니다.")

In [8]:
try:
    full_text = read_text_or_pdf_or_docx_or_excel(input_txt)
except Exception as e:
    print(f"파일 읽기 오류 발생: {e}")
    full_text = None

if full_text:
    result = get_requirements_from_text(full_text)
    print(result)
    with open("extracted_additional_requirements.json", "w", encoding="utf-8") as f:
        f.write(result)

[
    {
        "id": "FUNC-001",
        "type": "기능적",
        "description": "학습 이력 기반 AI 추천 기능",
        "detailed_description": "수강자의 학습 이력을 기반으로 강의를 추천하는 기능. 마이페이지에서 개인화된 추천 강의를 제안해야 함.",
        "acceptance_criteria": "세부 설계 시 정의",
        "module": "추천 시스템",
        "source_pages": "회의록",
        "raw_text_snippet": "수강자의 이력 기반으로 추천해주는 기능, 그거 한번 넣어보는 게 어떨까 싶습니다.",
        "status": "신규",
        "mod_reason": "회의 내용을 기반으로 신규 도출됨",
        "category_large": "학습",
        "category_medium": "학습 추천",
        "category_small": "AI 기반 학습 추천",
        "difficulty": "상",
        "importance": "상"
    },
    {
        "id": "FUNC-002",
        "type": "기능적",
        "description": "챗봇 기능",
        "detailed_description": "학습자들이 시스템을 처음 접할 때 도움을 주는 챗봇 기능. 기본적인 문의 처리를 챗봇으로 수행.",
        "acceptance_criteria": "세부 설계 시 정의",
        "module": "챗봇 시스템",
        "source_pages": "회의록",
        "raw_text_snippet": "전화나 이메일보단 요즘 챗봇으로 기본적인 문의처리하는 게 효율적인데, 그거도 한번 고려해보면 좋을 것 같아요.",
        "status